**Coursebook: Exploratory Data Analysis**

- Part 2 of Large-scale Data Processing with PySpark for DBS
- Durasi: 9 jam
- *Last Updated*: January 2024

---

## Training Objectives

Dalam workshop ini, kami akan berfokus pada serangkaian metode pembelajaran *Data Processing with PySpark*. Adapun beberapa *module* yang disediakan antara lain:

- **Frequency tables in PySpark SQL**
    - Higher dimensional table
    - Data aggregation
- **Dealing with untidy data**
    - Checking NaN values
    - Missing values treatment
    - Removing duplicates value
- **Calculation in PySpark**
    - Feature engineering in PySpark
    - Extract information from datetime

# Introduction

## Apa itu EDA?
Exploratory Data Analysis (EDA) adalah suatu proses untuk melakukan eksplorasi lebih jauh terhadap data, seperti:
- melihat struktur data, 
- melihat sebaran data,
- menyesuaikan bentuk tipe data untuk analisis lebih lanjut.

Ini juga dapat membantu menentukan apakah teknik statistik yang Anda pertimbangkan untuk analisis data sudah sesuai. Awalnya dikembangkan oleh matematikawan Amerika John Tukey pada 1970-an, teknik EDA terus menjadi metode yang banyak digunakan dalam proses penemuan data saat ini.

## Mengapa EDA penting?
Tujuan utama EDA adalah untuk membantu melihat data sebelum membuat asumsi apa pun.
- Ini dapat membantu mengidentifikasi kesalahan yang jelas,
- serta lebih memahami pola dalam data,
- mendeteksi outlier atau kejadian anomali,
- menemukan hubungan yang menarik antara variabel.

Ilmuwan data dan Analis Data dapat menggunakan analisis eksplorasi untuk:
- memastikan hasil yang mereka hasilkan valid dan berlaku untuk setiap hasil dan tujuan bisnis yang diinginkan,
- membantu pemangku kepentingan dengan mengonfirmasi bahwa mereka mengajukan pertanyaan yang tepat
- EDA selesai dan wawasan diambil, fitur-fiturnya kemudian dapat digunakan untuk analisis atau pemodelan data yang lebih canggih, termasuk pembelajaran mesin

# Problem Statement

🔻 Anda merupakan seorang data analyst yang diberi sebuah data tabular berformat `.csv`. Anda diminta untuk melakukan eksplorasi terhadap data tersebut hingga mendapatkan insight-insight bisnis yang dapat anda ceritakan kepada orang lain atau rekan Anda.

# Data Preparation

🔻 Hal pertama yang harus dilakukan adalah menghubungkan PySpark dengan sumber data yang akan diolah. Dalam hal ini kita tetap menggunakan `loan-missing.csv` berisi informasi peminjaman nasabah pada sebuah bank.

Untuk menghubungkan PySpark dengan sumber data, kita akan menggunakan cara yang telah dipelajari di couse sebelumnya mulai dari mengimport `pyspark` hingga pembacaan data menggunakan pyspark.

In [1]:
# import library
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
# membuat SparkSession
spark =SparkSession.builder.appName("DBS Indonesia").getOrCreate()

# membaca file CSV
loan = spark.read.csv('data_input/loan-missing.csv', header=True, inferSchema=True)

# menampilkan dataframe
loan.show()

+---+---------+------+----------+--------------+--------------+------------------+---------------+----------+----------+-----------+----------+--------+----------------+------------------+-----------+-----------------+--------------------+--------------+------------------+-------------+-----+---------+-----+-----------------+---------------+----------+-----------+------------+
|_c0|       id|  year|   issue_d|emp_length_int|home_ownership|home_ownership_cat|income_category|annual_inc|income_cat|loan_amount|      term|term_cat|application_type|           purpose|purpose_cat|interest_payments|interest_payment_cat|loan_condition|loan_condition_cat|interest_rate|grade|grade_cat|  dti|      total_pymnt|total_rec_prncp|recoveries|installment|      region|
+---+---------+------+----------+--------------+--------------+------------------+---------------+----------+----------+-----------+----------+--------+----------------+------------------+-----------+-----------------+--------------------+-

🔻 Lakukan investigasi awal untuk melihat struktur data terhadap object DataFrame dengan menggunakan method:
-  `df.count()` : untuk mendapatkan jumlah baris
-  `len(df.columns)`: untuk mendapatkan jumlah kolom

In [3]:
# jumlah baris
loan.count()

722423

In [15]:
# alternatif
loan.repartition(1).count()

722423

In [16]:
# alternatif, lebih lama
len(loan.toPandas())

722423

In [17]:
# convert to pandas
loan_df = loan.toPandas()
loan_df.head()

,_c0,id,year,issue_d,emp_length_int,home_ownership,home_ownership_cat,income_category,annual_inc,income_cat,...,grade,grade_cat,dti,total_pymnt,total_rec_prncp,recoveries,installment,region,year_new,month
0,0,1077501.0,2011.0,2011-12-01,10.0,RENT,1.0,Low,24000.0,1.0,...,B,2.0,27.65,5861.071414,5000.00,0.00,162.87,munster,2011.0,12.0
1,1,1077430.0,2011.0,2011-12-01,0.5,RENT,1.0,Low,30000.0,1.0,...,C,3.0,1.00,1008.710000,456.46,117.08,59.83,leinster,2011.0,12.0
2,2,1077175.0,2011.0,2011-12-01,10.0,RENT,1.0,Low,12252.0,1.0,...,C,3.0,8.72,3003.653644,2400.00,0.00,84.33,cannught,2011.0,12.0
3,3,1076863.0,2011.0,2011-12-01,10.0,RENT,1.0,Low,49200.0,1.0,...,C,3.0,20.00,12226.302210,10000.00,0.00,339.31,ulster,2011.0,12.0
4,4,1075358.0,2011.0,2011-12-01,1.0,RENT,1.0,Low,80000.0,1.0,...,B,2.0,17.94,3242.170000,2233.10,0.00,67.79,ulster,2011.0,12.0


In [18]:
loan_df.dtypes

_c0                       int32
id                      float64
year                    float64
issue_d                  object
emp_length_int          float64
home_ownership           object
home_ownership_cat      float64
income_category          object
annual_inc              float64
income_cat              float64
loan_amount             float64
term                     object
term_cat                float64
application_type         object
purpose                  object
purpose_cat             float64
interest_payments        object
interest_payment_cat    float64
loan_condition           object
loan_condition_cat      float64
interest_rate           float64
grade                    object
grade_cat               float64
dti                     float64
total_pymnt             float64
total_rec_prncp         float64
recoveries              float64
installment             float64
region                   object
year_new                float64
month                   float64
dtype: o

In [4]:
# jumlah kolom
len(loan.columns)

29

Selanjutnya perlu dilakukan inspeksi tipe data untuk memastikan tipe data setiap kolomnya telah tepat sehingga akan memudahkan dalam proses lanjutan dalam pengolahan data.

💡 Mengecek tipe data di PySpark: `df.printSchema()`

In [5]:
# code here
loan.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: double (nullable = true)
 |-- year: double (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- emp_length_int: double (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- home_ownership_cat: double (nullable = true)
 |-- income_category: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- income_cat: double (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- term: string (nullable = true)
 |-- term_cat: double (nullable = true)
 |-- application_type: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- purpose_cat: double (nullable = true)
 |-- interest_payments: string (nullable = true)
 |-- interest_payment_cat: double (nullable = true)
 |-- loan_condition: string (nullable = true)
 |-- loan_condition_cat: double (nullable = true)
 |-- interest_rate: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- grade_cat: double (nullable = true)
 |-- dti

## Perubahan Tipe Data (Datetime)

Ketika melakukan analisis data, merubah ke tipe data datetime penting dengan manfaat dapat ekstraksi informasi waktu, pengurutan datetime yang akurat serta filtering rentang waktu dengan mudah.

Untuk merubah kolom ke tipe data datetime:

```python
df = df.withColumn('issue_d', to_date(df['date_column'], format='dd/MM/yyyy'))
```

Notes: parameter `format` mengikuti format kolom tanggal awal.
- y = tahun (contoh: 2020; 20)
- M = bulan, numerik (contoh: 7; 07)
- L = bulan, string (contoh: Jul; July)
- d = tanggal (contoh: 28)

Simbol lebih lengkap: [Apache Spark Datetime Pattern](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)


**Format** yang tepat untuk `issue_d`?

>

In [6]:
# merubah ke date berdasarkan format
loan = loan.withColumn('issue_d', to_date('issue_d', format = 'dd/MM/yyyy'))

## Partisi Datetime

Setelah melakukan konversi tipe data menjadi bentuk `datetime`, kita dapat melakukan partisi untuk menggali informasi yang lebih spesifik seperti tahun, bulan, hari, dan jam. <br>

**Date component (numeric):**
- `year(df['date_column'])` -> partisi tahun
- `month(df['date_column'])` -> partisi bulan
- `day(df['date_column'])` -> partisi tanggal
- `quarter(df['date_column'])` -> partisi kuarter

**Date component (string):**
- `date_format(df['date_column'], format)` -> partisi sesuai [format](https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html)

**A. partisi `year(df['date_column'])`**

🔻Semisal kita diminta untuk menganalisis karakteristik nasabah berdasarkan **tahun**. Kita dapat membuat kolom baru dengan mengekstrak tahun dari kolom `issue_d`.

In [7]:
# Ekstrak Tahun
loan = loan.withColumn('year_new', year(loan['issue_d']))
loan.show(3)

+---+---------+------+----------+--------------+--------------+------------------+---------------+----------+----------+-----------+----------+--------+----------------+--------------+-----------+-----------------+--------------------+--------------+------------------+-------------+-----+---------+-----+-----------+---------------+----------+-----------+--------+--------+
|_c0|       id|  year|   issue_d|emp_length_int|home_ownership|home_ownership_cat|income_category|annual_inc|income_cat|loan_amount|      term|term_cat|application_type|       purpose|purpose_cat|interest_payments|interest_payment_cat|loan_condition|loan_condition_cat|interest_rate|grade|grade_cat|  dti|total_pymnt|total_rec_prncp|recoveries|installment|  region|year_new|
+---+---------+------+----------+--------------+--------------+------------------+---------------+----------+----------+-----------+----------+--------+----------------+--------------+-----------+-----------------+--------------------+--------------+

**B. partisi `month(df['date_column'])`**

🔻Semisal kita diminta untuk menganalisis karakteristik nasabah berdasarkan **bulan**. Kita dapat membuat kolom baru dengan mengekstrak **bulan** dari kolom `issue_d`.

In [8]:
# Ekstrak Bulan
loan = loan.withColumn('month', month(loan['issue_d']))

**C. partisi `date_format(df['date_column', format])`**

🔻Semisal kita diminta untuk membuat kolom baru dengan informasi nama hari seperti "Thu, 01-12-2011".

Sehingga format yang digunakan adalah `E, dd-MM-yyyy`, dimana:
- `E`: nama hari (contoh: Mon, Tue), gunakan format `EEEE` untuk nama hari tidak disingkat
- `d`: tanggal
- `M`: bulan
- `y`: tahun

In [9]:
# Ekstrak Hari
loan.withColumn('date_issue_new', date_format(loan['issue_d'], format='EEEE, dd/MMMM/yyyy')).\
select('date_issue_new').show(truncate=False)

+--------------------------+
|date_issue_new            |
+--------------------------+
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
|Thursday, 01/December/2011|
+--------------------------+
only showing top 20 rows



**END OF DAY 2**

---

**START OF DAY 3**

# Analisis Data

Dalam melakukan analisis data. Terdapat pertanyaan-pertanyaan seperti:

- Berapa jumlah customer kita yang memiliki kategori `interest_payment` HIGH?
- Berapa rata-rata income untuk setiap kategori di `loan_condition`
- Berapa total pinjaman pada tahun 2011?
- dan lain-lain.

Pertanyaan-pertanyaan tersebut dapat dijawab dengan melakukan **agregasi tabel**. Agregasi tabel adalah tabel hasil pengelompokkan dengan nilai-nilai statistik seperti jumlah, rata-rata, kemunculan, dan lain sebagainya.

Untuk melakukan agregasi table di `pyspark` terdapat 2 cara, antara lain:
- `df.crosstab()`
- `df.groupby()`

## Crosstab()

🔻**Contoh kasus:** sebagai data analis, kita diminta untuk menghitung jumlah customer `BAD LOAN` yang diberikan interest `HIGH`.

Untuk menjawab pertanyaan ini, kita dapat menggunakan `.crosstab()`. Fungsi `.crosstab()` di pyspark untuk menghitung jumlah kemunculan dari 2 kolom kategori yang berbeda.

**Syntax:**

```python
df.crosstab("kategori_1", "kategori_2")
```

Untuk menjawab pertanyaan tersebut, mari memilih kolom yang diperlukan.
> `interest_payment`, `loan_condition`

In [ ]:
# Melihat kolom-kolom yang diperlukan


In [ ]:
# melakukan crosstab


**Insight:**

`crosstab()` hanya dapat untuk menghitung frekuensi dari 2 kolom kategori. Namun bagaimana jika yang ingin dihitung untuk 1 kategori? atau bahkan menghitung rata-rata nilai numerik dari kolom kategori tertentu?

Permasalahan di atas dapat diselesaikan oleh `groupBy()`

## groupBy()

Untuk membuat tabel agregasi, kita dapat menggunakan fungsi `groupBy()` diikuti dengan fungsi agregasi yang mau kita hitung.

🔻 Sebagai data analis, kita diminta untuk menghitung customer untuk masing-masing kategori di `home_ownership`.

**Syntax** `groupBy()`

```python
df.groupBy('kolom_kategori').FUNC()
```

Dimana `.FUNC()` bisa berupa:
- `.count()`: untuk menghitung jumlah kemunculan
- `.sum()`: jumlah semua kolom numerik / `.sum('kolom_numerik')` untuk jumlah kolom tertentu.
- `.avg()`: rata-rata semua kolom numerik / `.avg('kolom_numerik')` untuk jumlah kolom tertentu.


In [ ]:
# code here


🔻 Sebagai data analis, kita diminta untuk menghitung rata-rata `loan_amount` dan `annual_inc` untuk customer dengan kondisi loan BAD dan GOOD.

In [ ]:
# code here


**Insight:**

Dari hasil di atas, kita akan mendapatkan semua rata-rata dari kolom numerik. Bagaimana jika kita ingin menghitung salah satu kolom saja? semisal rata-rata `annual_inc` untuk customer GOOD loan dan BAD loan.

In [ ]:
# code here


## agg()

Misalkan kita ingin membuat tabel agregasi dengan `.FUNC()` yang berbeda-beda untuk masing-masing kolom berupa:
- rata-rata untuk `annual_inc`
- jumlah untuk `loan_amount` dan `total_pymnt`

Untuk mendapatkan hasil tersebut, kita harus melakukan chaining `groupBy` dengan method `agg()`. Kita harus menyertakan mapping (**dictionary**) untuk setiap kolom dengan fungsi agregasinya seperti berikut:

**Syntax:**
```python
df.agg({
    'nama_kolom': 'fungsi_agregasi',
    'nama_kolom2': 'fungsi_agregasi'
})
```

In [ ]:
# code here


**🧠 Dive Deeper**

1. Berapakah rata-rata `installment` untuk masing-masing `purpose` pinjaman?

In [ ]:
# code here


2. Berapakah rata-rata `annual_inc` untuk setiap `income_category` dan jenis `home_ownership`?

In [ ]:
# code here


# Untidy Data

Dalam melakukan pengolahan data, tidak semua data yang kita miliki adalah data yang "tidy". Ada kemungkinan bahwa data kita memiliki nilai kosong (NULL), memiliki nilai yang berulang, dan memiliki nilai yang tidak sesuai dengan nilai variable yang seharusnya (misal usia memiliki nilai minus). Untuk mengatasi hal tersebut, kita dapat melakukan beberapa metode penanganan pada data yang hilang (missing value) atau data yang duplikat (duplicates value). 

Penanganan data yang hilang atau data yang duplikat menjadi hal yang penting guna memberikan analisa dan insight yang lebih akurat.

## Inconsistency Values

Realitanya, ketika kita melakukan proses pengolahan data, kendala yang sering sekali di hadapi adalah value yang tidak konsisten dalam data. Tidak konsisten yang dimaksudkan disini adalah, terdapat beberapa karakter atau spasi berlebih dan juga penggunaan huruf *lowercase* dan *uppercase* yang tidak sesuai. Pada tahapan ini, kita akan mencoba untuk melakukan proses handling pada data yang tidak konsisten tersebut.

**Membuat Dummy Data**

In [ ]:
data = [([" Dyah Nurlita ",'1994/12/30']), ([" Samuel   Chan ",'1995/11/30']), (["Irfan Rahman",'1992/11/14'])]
columns = ["Name","Date"]
df = spark.createDataFrame(data, columns)
df.show()

In [ ]:
df = df.withColumn('Date', to_date('Date', format='dd/MM/yyyy'))
df.show()

### Menghilangkan Spasi Berlebih

1. Menghilangkan spasi berlebih yang terdapat di awal dan di akhir kalimat

In [ ]:
# code here



2. Menghilangkan spasi di semua text

In [ ]:
# code here



3. Mengganti spasi dengan karakter tertentu

In [ ]:
# code here



### *Uppercase* dan *Lowercase*

**Menggubah text menjadi *uppercase***

In [ ]:
# code here



**Mengubah text menjadi *lowercase***

In [ ]:
# code here



## Missing Value

### Checking Missing Value

Untuk mengecek missing values apakah terdapat pada setiap kolom dapat menggunakan 2 cara berikut:
- `df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()`

Dapat terlihat bahwa semua kolom memiliki missing value. 

Untuk menghitung berapa jumlah missing value di setiap kolomnya dapat menggunakan cara 2.

In [ ]:
# cara hapus missing values
loan.select([count(when(col(c).isNull(), c)).alias(c) for c in loan.columns]).show()

### Treatment Missing Values

Beberapa cara umum untuk menangani missing values:

1. Hapus baris atau kolom: Menggunakan metode `.dropna()`
2. Imputasi nilai NA dengan sebuah nilai
3. Tetap mempertahankan missing value

### Remove Missing Values

Membuang nilai missing dilakukan dengan fungsi `dropna()`. Secara default, `dropna()` akan menghapus semua baris yang terdapat nilai missingnya. Adapun parameter `.dropna()` antara lain:

- `.dropna(how='any')`: hapus baris apabila memiliki **minimal 1 kolom** nilai missing value

- `.dropna(how='all')`: harus baris apabila memiliki **semua kolom** nilai missing

- `.dropna(thresh=...)`: hapus baris apabila nilai **non-missing** < `thresh` 

In [ ]:
# how="all"


**Tresh** : Paling tidak ada jumlah kolom sebanyak `thresh` yang terisi

In [ ]:
# thresh=28


> Terlihat index 5, 7, dan 12 telah terhapus karena jumlah kolom yang notnull di bawah `threshold`. Index 8 masih memiliki missing value tapi tidak di-drop.

In [ ]:
# how="any"


> Terlihat bahwa semua baris yang mengandung missing value akan dihapus. Contoh 5,7,8,12.

### Impute Missing Values

Kita akan melakukan imputasi terhadap data yang mengandung missing value, menggunakan metode `.fillna()`

💡 **Tips** untuk imputasi:

Untuk kolom numerik:

- Isi menggunakan pusat data seperti `mean` atau `median`

Untuk kolom kategorikal:

- Menggunakan `NA` sebagai salah satu dari kategori
- Isi menggunakan pusat data (mode)

Untuk kolom datetime:

- Menggunakan metode [`bfill`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.bfill.html): melakukan imputasi dari baris bawah ke atas
- Menggunakan metode [`ffill`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.ffill.html): melakukan imputasi dari baris atas ke bawah

**Case 1**
Misalkan kita ketahui bahwa nilai missing pada `purpose` menjadi "other". `income_category` menjadi "Low" serta `income_cat` menjadi 1.0 dengan asumsi customer yang datanya tidak lengkap maka dianggap menjadi berpenghasilan rendah.

In [ ]:
# code here



**Case 2** Misalkan kita ingin mengisi **interest_rate** yang NULL dengan nilai rata-rata interest rate dari seluruh customer.

In [ ]:
# Mencari rata-rata untuk interest_rate


In [ ]:
# Mengisi interest rate dengan nilai rata-rata


### [Opt] Imputer

Selain menggunakan cara-cara di atas, pyspark menyediakan metode khusus untuk melakukan imputasi missing value dengan bantuan `Imputer()`. Dengan menggunakan `Imputer()` kita bisa langsung memilih nilai apa yang mau kita hitung dari 3 nilai yaitu **mean, median, dan modus**. Hal yang perlu diperhatikan adalah penggunaan `Imputer()` hanya bisa digunakan pada data atau kolom numerik saja

In [ ]:
from pyspark.ml.feature import Imputer

In [ ]:
imputer = Imputer(
    strategy='mean',
    inputCols=['loan_amount'],
    outputCols=['loan_amount']
)
imputer.fit(loan).transform(loan).show()

Pada kelas ini, kita akan drop semua baris yang memiliki missing values pada data kita. Maka dari itu digunakan `dropna()` dengan nilai default dan akan menyimpan ke dataframe `loan_clean`

In [ ]:
loan_clean = loan.dropna()

loan_clean.show()

In [ ]:
loan.count() - loan_clean.count()

Terdapat 10rb dari 722rb data yang didrop karena memiliki missing value.

## Duplicated Data

Untuk menghilangkan duplikat pada dataframe, kita bisa menggunakan fungsi `.dropDuplicates()` atau hanya mengambil nilai uniknya saja.


## Checking Duplicate

Pertama, mari kita lihat baris-baris yang memiliki nilai duplikat di data kita. Untuk melihatnya dapat menggunakan:

**Syntax:**
```python
df.exceptAll(df.dropDuplicates()).show()
```

In [ ]:
# code here



## Handling Duplicate

Pada data kita masih belum terdapat nilai duplikat. Namun, jika nantinya terdapat nilai duplikat, maka dapat drop nilai duplicated dapat `.dropDuplicates()`.

**Syntax:**
```python
df.dropDuplicates()
```

In [ ]:
# code here



Fungsi tersebut akan mempertimbangkan semua kolom untuk melihat apakah ada nilai yang duplikat atau tidak. 
Apabila kita hanya ingin mempertimbangkan kolom tertentu saja maka bisa menambahkan parameter subset

**Syntax:**
```
df.dropDuplicates(subset = ['kolom_1', 'kolom_2'])
```

**Case:** Semisal kita ingin membuang nilai baris yang kolom `id`, `issue_d`, dan `loan_amount`nya secara bersamaan bernilai duplikat dengan asumsi terdapat duplikat input untuk customer yang sama di hari yang sama.

In [ ]:
# code here



⚠️ Warning: Duplikat dapat berarti hal yang berbeda dari sudut pandang data dan sudut pandang analis bisnis. Anda harus ekstra berhati-hati apakah data duplikat memang merupakan karakteristik dari data Anda, atau apakah itu merupakan sebuah kesalahan input data berdasarkan logika bisnisnya.

# Bonus

## Substring

In [28]:
data_new = [(["Dyah Nurlita"]), (["Triani Narita"]), (["Victor Nugraha"])]
columns = ["Name"]
df_new = spark.createDataFrame(data_new, columns)
df_new.show()

+--------------+
|          Name|
+--------------+
|  Dyah Nurlita|
| Triani Narita|
|Victor Nugraha|
+--------------+



1. Mengambil 4 karakter pertama dari kolom Nama

In [31]:
df_new = df_new.withColumn('First_4_Chars', col("Name").substr(1, 4))

df_new.show()

+--------------+-------------+
|          Name|First_4_Chars|
+--------------+-------------+
|  Dyah Nurlita|         Dyah|
| Triani Narita|         Tria|
|Victor Nugraha|         Vict|
+--------------+-------------+



2. Mengambil 4 karakter terakhir dari kolom Nama

In [34]:
df_new = df_new.withColumn('Last_4_Chars', expr("substr(Name, length(Name)-3, 4)"))
df_new.show()

+--------------+-------------+------------+
|          Name|First_4_Chars|Last_4_Chars|
+--------------+-------------+------------+
|  Dyah Nurlita|         Dyah|        lita|
| Triani Narita|         Tria|        rita|
|Victor Nugraha|         Vict|        raha|
+--------------+-------------+------------+



# Inclass Question

In [12]:
trial = spark.read.csv('data_input/trial.txt', sep = ';',header=True, inferSchema=True)

trial.show()

+------+------+------+
|kolom1|kolom2|kolom3|
+------+------+------+
|     1|     2|     3|
|     4|     5|     6|
+------+------+------+

